# analyze_bias.ipynb
Quantitative + qualitative analyses on `./results/responses.csv`.

**Outputs:**
- `analysis/analysis_summary.json`
- `analysis/chi_square_results.json`

This notebook version mirrors the Python script and is safe to run inside Jupyter.

In [7]:
import csv, json, re, math
from pathlib import Path
from collections import Counter, defaultdict

# Use current directory for Jupyter friendliness
ROOT = Path('.')
RESP = ROOT / 'results' / 'responses.csv'
OUT_DIR = ROOT / 'analysis'
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Minimal sentiment lexicon (extend as needed)
POS = set("improve improving growth potential breakout strong efficient efficiency excellent good great positive opportunity opportunities upside".split())
NEG = set("poor struggle struggling weak decline declining worse worst negative risk downside issue issues problem problems".split())

PLAYER_PATTERN = re.compile(r"(player\s+[A-Z]|Player\s+[A-Z])", re.I)

def sentiment_score(text):
    tokens = re.findall(r"[A-Za-z']+", text.lower())
    pos = sum(t in POS for t in tokens)
    neg = sum(t in NEG for t in tokens)
    return pos - neg

def chi_square(observed_counts_by_group):
    """
    observed_counts_by_group: dict[group] -> Counter(category->count)
    Returns chi2, dof, expected matrix for reference.
    """
    groups = list(observed_counts_by_group.keys())
    cats = sorted({c for g in groups for c in observed_counts_by_group[g]})
    # Build table
    table = []
    row_sums = []
    for g in groups:
        row = [observed_counts_by_group[g].get(c,0) for c in cats]
        table.append(row)
        row_sums.append(sum(row))
    col_sums = [sum(row[i] for row in table) for i in range(len(cats))]
    total = sum(col_sums)
    chi2 = 0.0
    for r,g in enumerate(groups):
        for i,c in enumerate(cats):
            expected = (row_sums[r]*col_sums[i])/total if total>0 else 0
            observed = table[r][i]
            if expected > 0:
                chi2 += (observed-expected)**2/expected
    dof = (len(groups)-1)*(len(cats)-1)
    return {"chi2": chi2, "dof": dof, "groups": groups, "categories": cats, "table": table, "row_sums": row_sums, "col_sums": col_sums, "total": total}

print('✅ Setup complete. Ready to analyze results/responses.csv')


✅ Setup complete. Ready to analyze results/responses.csv


In [8]:
# Run analysis
if not RESP.exists():
    print("No responses.csv found. Add responses via run_experiment (notebook or .py) and re-run this cell.")
else:
    rows = []
    with open(RESP, encoding='utf-8') as f:
        for i, row in enumerate(csv.DictReader(f)):
            rows.append(row)

    # Buckets
    sentiment_by_h = defaultdict(list)
    mentions_by_h = defaultdict(Counter)
    reco_bucket_by_h = defaultdict(Counter)

    for r in rows:
        h = r.get('hypothesis_id','')
        txt = r.get('response_text','')
        # sentiment
        sentiment_by_h[h].append(sentiment_score(txt))
        # mentions
        for m in PLAYER_PATTERN.findall(txt):
            k = m.strip().title()
            mentions_by_h[h][k] += 1
        # reco types
        low = txt.lower()
        if 'offense' in low: reco_bucket_by_h[h]['offense'] += 1
        if 'defense' in low: reco_bucket_by_h[h]['defense'] += 1
        if 'team' in low:    reco_bucket_by_h[h]['team'] += 1
        if re.search(r"\bplayer\b", low): reco_bucket_by_h[h]['individual'] += 1

    summary = {
        'n_rows': len(rows),
        'sentiment_mean_by_hypothesis': {h: (sum(v)/len(v) if v else 0.0) for h,v in sentiment_by_h.items()},
        'mentions_by_hypothesis': {h: dict(c) for h,c in mentions_by_h.items()},
        'recommendation_buckets_by_hypothesis': {h: dict(c) for h,c in reco_bucket_by_h.items()},
    }

    pairs = [("H1_framing_negative","H1_framing_positive"),
             ("H3_confirmation_neutral","H3_confirmation_primed")]
    chi_results = {}
    for a,b in pairs:
        observed = {
            a: reco_bucket_by_h.get(a, Counter()),
            b: reco_bucket_by_h.get(b, Counter())
        }
        chi_results[f"{a}_vs_{b}"] = chi_square(observed)

    (OUT_DIR / 'analysis_summary.json').write_text(json.dumps(summary, indent=2))
    (OUT_DIR / 'chi_square_results.json').write_text(json.dumps(chi_results, indent=2))
    print('Wrote analysis/analysis_summary.json and analysis/chi_square_results.json')


Wrote analysis/analysis_summary.json and analysis/chi_square_results.json


In [9]:
# Quick peek at outputs (if created)
from pathlib import Path
out1 = Path('analysis/analysis_summary.json')
out2 = Path('analysis/chi_square_results.json')
if out1.exists():
    print('analysis_summary.json:')
    txt = out1.read_text()
    print(txt[:1000] + ('...' if len(txt)>1000 else ''))
else:
    print('analysis_summary.json not found yet.')
if out2.exists():
    print('\nchi_square_results.json:')
    txt2 = out2.read_text()
    print(txt2[:1000] + ('...' if len(txt2)>1000 else ''))
else:
    print('chi_square_results.json not found yet.')


analysis_summary.json:
{
  "n_rows": 10,
  "sentiment_mean_by_hypothesis": {
    "": 0.0,
    "H1": 0.5,
    "H2": 0.0,
    "H3": 0.0,
    "H4": 0.0
  },
  "mentions_by_hypothesis": {
    "H1": {
      "Player A": 2
    }
  },
  "recommendation_buckets_by_hypothesis": {
    "H1": {
      "individual": 2
    }
  }
}

chi_square_results.json:
{
  "H1_framing_negative_vs_H1_framing_positive": {
    "chi2": 0.0,
    "dof": -1,
    "groups": [
      "H1_framing_negative",
      "H1_framing_positive"
    ],
    "categories": [],
    "table": [
      [],
      []
    ],
    "row_sums": [
      0,
      0
    ],
    "col_sums": [],
    "total": 0
  },
  "H3_confirmation_neutral_vs_H3_confirmation_primed": {
    "chi2": 0.0,
    "dof": -1,
    "groups": [
      "H3_confirmation_neutral",
      "H3_confirmation_primed"
    ],
    "categories": [],
    "table": [
      [],
      []
    ],
    "row_sums": [
      0,
      0
    ],
    "col_sums": [],
    "total": 0
  }
}
